In [1]:
import os
import sys
module_path = os.path.join("../../KeyPoint-Analysis/KPG")
sys.path.insert(0, module_path)

In [2]:
from rouge_setbase import preprocess_dataset, compute_rouge, compute_rouge_max
from softF1 import softevaluation

In [3]:
import pandas as pd
import ast

# Import Data

## Prediction Data

In [3]:
# import spacy
# nlp = spacy.load('en_core_web_lg')

In [4]:
prediction_df = pd.read_pickle(f"../../PRIMER/Topic_Clustering_Simple/space_data_aspect_clustering_by_business_0.55/gpt_generation_v3_single_aspect/kp_extract_clustering_kp_generation_0.55_merge_thres_raw.pkl")
prediction_df.insert(5, 'coverage', prediction_df['comments'].str.len())
prediction_df = prediction_df[~prediction_df['generated_kp'].str.contains('comments')]
prediction_df = prediction_df.iloc[:,:6]
prediction_df = prediction_df.rename(columns={'cluster_sentiment': 'sentiment'})

In [5]:
prediction_df

,topic,entity_id,entity_name,sentiment,generated_kp,coverage
0,Hotel,156564,Wedgewood Hotel & Spa,negative,Depressing and unhelpful atmosphere.,5
1,Hotel,156564,Wedgewood Hotel & Spa,negative,Disturbing construction noise.,4
2,Hotel,156564,Wedgewood Hotel & Spa,negative,In need of renovation.,9
3,Hotel,156564,Wedgewood Hotel & Spa,negative,Inexperienced staff with limited knowledge.,4
4,Hotel,156564,Wedgewood Hotel & Spa,negative,Issue with the toilet in the suite.,2
...,...,...,...,...,...,...
369,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Perfect evening and morning.,5
370,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Positive experience at the Erwin.,7
371,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Spacious and well-equipped suites.,13
372,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Stunning ocean views.,34


In [6]:
# prediction_df['num_of_token'] = prediction_df['generated_kp'].apply(lambda x: len([token for token in nlp(x) if token.pos_ not in ['PUNCT', 'SPACE']]))

In [7]:
prediction_df.shape

(358, 6)

In [8]:
# prediction_df = prediction_df[(prediction_df['num_of_token'] >= 3) & (prediction_df['num_of_token'] <= 5)]
# prediction_df = prediction_df[(prediction_df['num_of_token'] >= 3) & (prediction_df['num_of_token'] <= 6)]

In [9]:
prediction_df.shape

(358, 6)

In [10]:
# min_coverage = 1
# min_coverage = 5
# min_coverage = 10
min_coverage = 15
prediction_df = prediction_df[prediction_df['coverage'] > min_coverage]
prediction_df

,topic,entity_id,entity_name,sentiment,generated_kp,coverage
7,Hotel,156564,Wedgewood Hotel & Spa,negative,Outdated and cramped room.,19
17,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,96
18,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious suites.,24
21,Hotel,156564,Wedgewood Hotel & Spa,positive,Comfortable and luxurious beds.,17
23,Hotel,156564,Wedgewood Hotel & Spa,positive,Delicious cookies provided during turndown service.,19
...,...,...,...,...,...,...
355,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Excellent location with great service.,50
357,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Friendly and helpful staff.,44
364,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Highly recommended hotel with great experience.,89
372,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Stunning ocean views.,34


## SPACE Summary Data

In [23]:
space_df = pd.read_json("../../../Quantized_Transformer_SPACE/space/space_summ.json")

In [24]:
# space_df

In [25]:
data = space_df.to_json(orient='records')
data = ast.literal_eval(data)

In [26]:
summary_df = pd.json_normalize(data, max_level=1)

In [27]:
summary_df = summary_df.melt(id_vars=['entity_id', 'entity_name', 'reviews'], var_name='aspects', value_name='gold_summary')

In [28]:
summary_df.explode(['gold_summary'])['gold_summary'].iloc[0]

"The staff are friendly and exceptional. Every room (lobby included) was very clean. They are spacious, very quiet, and come with a coffee maker. Though, the rooms are outdated in decor. The hotel itself is conveniently close to the airport and restaurants. There's a chocolate-chip cookie at arrival, and for the prices, the experience is a good value."

In [29]:
summary_df = summary_df.explode(['gold_summary'])

In [19]:
# summary_df

In [34]:
import spacy
# spacy.require_gpu()
nlp = spacy.load('en_core_web_lg')

In [21]:
summary_df['gold_summary_sentences'] = summary_df['gold_summary'].apply(lambda x: [str(sentence).strip() for sentence in nlp(x).sents])

In [22]:
summary_df['gold_summary_sentences']

0      [The staff are friendly and exceptional., Ever...
0      [Service was exceptional and the quality was g...
0      [All the staff was exceptionally helpful, cour...
1      [The staff was friendly, helpful, and very qui...
1      [Excellent service!, The staff were pretty fri...
                             ...                        
348    [The inhouse restaurant was healthy and superb...
348    [The food at Hash, the restaurant just off of ...
349    [The option to cook at an in-room kitchen or t...
349    [Some rooms have kitchens so food was easy to ...
349    [This hotel located in wine country has good f...
Name: gold_summary_sentences, Length: 1050, dtype: object

In [23]:
summary_df = summary_df.explode(['gold_summary_sentences'])

In [24]:
summary_df

,entity_id,entity_name,reviews,aspects,gold_summary,gold_summary_sentences
0,100597,Doubletree by Hilton Seattle Airport,"[{'review_id': 'UR59977476', 'sentences': ['We...",summaries.general,The staff are friendly and exceptional. Every ...,The staff are friendly and exceptional.
0,100597,Doubletree by Hilton Seattle Airport,"[{'review_id': 'UR59977476', 'sentences': ['We...",summaries.general,The staff are friendly and exceptional. Every ...,Every room (lobby included) was very clean.
0,100597,Doubletree by Hilton Seattle Airport,"[{'review_id': 'UR59977476', 'sentences': ['We...",summaries.general,The staff are friendly and exceptional. Every ...,"They are spacious, very quiet, and come with a..."
0,100597,Doubletree by Hilton Seattle Airport,"[{'review_id': 'UR59977476', 'sentences': ['We...",summaries.general,The staff are friendly and exceptional. Every ...,"Though, the rooms are outdated in decor."
0,100597,Doubletree by Hilton Seattle Airport,"[{'review_id': 'UR59977476', 'sentences': ['We...",summaries.general,The staff are friendly and exceptional. Every ...,The hotel itself is conveniently close to the ...
...,...,...,...,...,...,...
349,2516165,Roman Spa Hot Springs Resort,"[{'review_id': 'UR20765890', 'sentences': ['We...",summaries.food,The option to cook at an in-room kitchen or to...,The option to cook at an in-room kitchen or to...
349,2516165,Roman Spa Hot Springs Resort,"[{'review_id': 'UR20765890', 'sentences': ['We...",summaries.food,The option to cook at an in-room kitchen or to...,The wonderful food at the hotel is also very g...
349,2516165,Roman Spa Hot Springs Resort,"[{'review_id': 'UR20765890', 'sentences': ['We...",summaries.food,Some rooms have kitchens so food was easy to c...,Some rooms have kitchens so food was easy to c...
349,2516165,Roman Spa Hot Springs Resort,"[{'review_id': 'UR20765890', 'sentences': ['We...",summaries.food,Some rooms have kitchens so food was easy to c...,There were also grills for barbecuing.


### Sentiment Analysis

In [25]:
from datasets import Dataset

In [26]:
sent_datasets = Dataset.from_pandas(summary_df)
sent_datasets

Dataset({
    features: ['entity_id', 'entity_name', 'reviews', 'aspects', 'gold_summary', 'gold_summary_sentences', '__index_level_0__'],
    num_rows: 2722
})

#### Tokenization

In [27]:
sent_datasets

Dataset({
    features: ['entity_id', 'entity_name', 'reviews', 'aspects', 'gold_summary', 'gold_summary_sentences', '__index_level_0__'],
    num_rows: 2722
})

In [28]:
model_checkpoint = "roberta-large"

In [29]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

In [30]:
sent_datasets.set_format(None)

In [31]:
def tokenize_function(examples):
    return tokenizer(examples["gold_summary_sentences"], padding="max_length", truncation=True)

tokenized_datasets = sent_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/2722 [00:00<?, ? examples/s]

In [32]:
tokenized_datasets

Dataset({
    features: ['entity_id', 'entity_name', 'reviews', 'aspects', 'gold_summary', 'gold_summary_sentences', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2722
})

In [33]:
remove_cols = tokenized_datasets.column_names
tokenized_datasets = tokenized_datasets.remove_columns(list(set(remove_cols) - set(['input_ids', 'attention_mask'])))

In [34]:
tokenized_datasets.set_format("torch")

#### Load Model

In [35]:
src_model_path = "../../../argmining-21-review-sentiment-keypoint-analysis/code/src-ipynb"
pretrained_checkpoint =  src_model_path + "/model/roberta-large-finetuned-yelp-sentiment-classification/checkpoint-189543/"

In [36]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(pretrained_checkpoint, num_labels=3)

In [37]:
model_name = model_checkpoint.split("/")[-1]
model_name

'roberta-large'

**NOTE: HAVE TO SET 'LEARNING RATE' to 1e-5 (default 5e-5) for the any large model (RoBERTA large), otherwise the trained result will be biased and tend to predict only one label**

In [38]:
from transformers import TrainingArguments, Trainer

batch_size = 64
logging_steps = 2000
training_args = TrainingArguments(output_dir=f"model/{model_name}-finetuned-yelp-sentiment-classification",
                                  logging_dir=f"log/{model_name}-finetuned-yelp-sentiment-classification",
                                  evaluation_strategy="epoch",
#                                   evaluation_strategy="steps",
                                  save_strategy="epoch",
                                  learning_rate=1e-5,
                                  num_train_epochs=3,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  fp16=True,
                                  logging_steps=logging_steps)

2024-01-23 01:13:07.143699: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 01:13:07.165291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 01:13:07.165317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 01:13:07.166033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 01:13:07.169924: I tensorflow/core/platform/cpu_feature_guar

In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
)

Using amp half precision backend


#### Predict

In [41]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2722
})

In [42]:
new_predictions = trainer.predict(tokenized_datasets)

***** Running Prediction *****
  Num examples = 2722
  Batch size = 64


In [43]:
import numpy as np
predict = np.argmax(new_predictions.predictions, axis=-1)

In [44]:
len(predict)

2722

In [45]:
summary_df['polarity_score'] = predict

In [46]:
summary_df['polarity_score'].value_counts()

polarity_score
2    1684
1    1001
0      37
Name: count, dtype: int64

In [47]:
summary_df[summary_df['polarity_score'] == 1].iloc[7]['gold_summary_sentences']

'The location was good, very close to the Bayside area and just a $15 ride to Lincoln Rd\\/South Beach.'

In [48]:
# summary_df[(summary_df['polarity_score'] == 1) & (kp_df['num_of_token'] < 8)]['key_point'].tolist()

In [49]:
summary_df[summary_df['polarity_score'] == 0]['gold_summary_sentences']

7      When a hotel strives for excellence in costume...
15     Being freshly done up, the hotel rooms are cle...
31     There are issues with the shower and bath, and...
32     This high-rise hotel has two elevators and one...
32          The Whiskey Bar and Grill was disappointing.
32     The building has two elevators but one wasn't ...
40     They make many time-consuming special arrangem...
42     Restaurant in hotel is good for breakfast but ...
50     Some things that need work is the bad lighting...
51             In the bathrooms there were clean towels.
54                  The bathroom was very tired looking.
58     They were recently decorated and in a style si...
61           Some small black hairs were found on a bed.
64                        Even the bathrooms were roomy.
67                                 Carpeted rooms smell.
74     Beds were squeaky and rooms by the pool at nig...
74                                No fridge in the room.
79                 The bathroom

In [50]:
summary_df['polarity'] = summary_df['polarity_score'].apply(lambda x: 1 if x >= 1 else 0)
summary_df['polarity'].value_counts()

polarity
1    2685
0      37
Name: count, dtype: int64

In [51]:
summary_df['sentiment'] = summary_df['polarity'].apply(lambda x: "positive" if x == 1 else "negative")

In [68]:
summary_df.to_pickle("./summary_df_sentiment_analyzed.pkl")

In [9]:
summary_df = pd.read_pickle("./summary_df_sentiment_analyzed.pkl")

In [35]:
summary_df['num_of_token'] = summary_df['gold_summary_sentences'].apply(lambda x: len([token for token in nlp(x) if token.pos_ not in ['PUNCT', 'SPACE']]))

In [38]:
summary_df = summary_df[summary_df['num_of_token'] < 20]

## Merge

In [11]:
summary_df = pd.read_pickle("./summary_df_sentiment_analyzed.pkl")
# summary_df = pd.read_pickle("./summary_df_sentiment_analyzed_spacy.pkl")

In [12]:
print(summary_df.shape)
summary_df = summary_df[summary_df['aspects'] != 'summaries.general']
summary_df.shape

(2722, 9)


(1860, 9)

In [13]:
merged_df = prediction_df.merge(summary_df, on=['entity_id', 'entity_name', 'sentiment'], how='left')
merged_df

topic  entity_id                         entity_name sentiment  \
0     Hotel     156564               Wedgewood Hotel & Spa  negative   
1     Hotel     156564               Wedgewood Hotel & Spa  positive   
2     Hotel     156564               Wedgewood Hotel & Spa  positive   
3     Hotel     156564               Wedgewood Hotel & Spa  positive   
4     Hotel     156564               Wedgewood Hotel & Spa  positive   
...     ...        ...                                 ...       ...   
2331  Hotel    2516007  Hotel Erwin, a Joie de Vivre hotel  positive   
2332  Hotel    2516007  Hotel Erwin, a Joie de Vivre hotel  positive   
2333  Hotel    2516007  Hotel Erwin, a Joie de Vivre hotel  positive   
2334  Hotel    2516007  Hotel Erwin, a Joie de Vivre hotel  positive   
2335  Hotel    2516007  Hotel Erwin, a Joie de Vivre hotel  positive   

                            generated_kp  coverage  \
0             Outdated and cramped room.        19   
1       Beautiful and spacious bathroom.        96   
2       Beautiful and spacious bathroom.        96   
3       Beautiful and spacious bathroom.        96   
4       Beautiful and spacious bathroom.        96   
...                                  ...       ...   
2331  The rooftop bar has amazing views.        56   
2332  The rooftop bar has amazing views.        56   
2333  The rooftop bar has amazing views.        56   
2334  The rooftop bar has amazing views.        56   
2335  The rooftop bar has amazing views.        56   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [53]:
# merged_df.to_pickle("./KPA_GPT_Merged_SPACE_Eval_3_8_tokens.pkl")

# Evaluation

In [3]:
merged_df = pd.read_pickle("./KPA_GPT_Merged_SPACE_Eval_3_8_tokens.pkl")

In [14]:
pd.set_option('max_colwidth', 500)

In [15]:
merged_df = merged_df.dropna()

In [16]:
from softF1 import *

In [17]:
merged_df = merged_df[['topic', 'entity_id', 'entity_name', 'sentiment', 'generated_kp', 'gold_summary_sentences']]

In [18]:
merged_df = merged_df.rename(columns={
    'generated_kp': 'key_point',
    'topic': 'category',
    'entity_name': 'topic',
    'sentiment': 'stance',
    'gold_summary_sentences': 'key_point_given'
})

In [19]:
merged_df

,category,entity_id,topic,stance,key_point,key_point_given
1,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The rooms are beautiful, spacious, and quite."
2,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers."
3,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The beds are amazingly comfortable, with top end linens."
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,These quiet and spacious and luxurious rooms had wonderful European feel with great showers.
5,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortable stay."
...,...,...,...,...,...,...
2331,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort."
2332,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash."
2333,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty."
2334,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast."


## BARTScore

### SoftPrecision

In [19]:
softp_data = merged_df.groupby(['category', 'entity_id', 'topic', 'stance', 'key_point'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point_given'].tolist())])).reset_index(name='multi_cands')
softp_data

,category,entity_id,topic,stance,key_point,multi_cands
0,Hotel,156564,Wedgewood Hotel & Spa,positive,A wonderful and quiet place.,"The rooms are beautiful, spacious, and quite.=Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.=The beds are amazingly comfortable, with top end linens.=These quiet and spacious and luxurious rooms had wonderful European feel with great showers.=Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortab..."
1,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The rooms are beautiful, spacious, and quite.=Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.=The beds are amazingly comfortable, with top end linens.=These quiet and spacious and luxurious rooms had wonderful European feel with great showers.=Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortab..."
2,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious suites.,"The rooms are beautiful, spacious, and quite.=Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.=The beds are amazingly comfortable, with top end linens.=These quiet and spacious and luxurious rooms had wonderful European feel with great showers.=Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortab..."
3,Hotel,156564,Wedgewood Hotel & Spa,positive,Comfortable amenities and relaxing atmosphere.,"The rooms are beautiful, spacious, and quite.=Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.=The beds are amazingly comfortable, with top end linens.=These quiet and spacious and luxurious rooms had wonderful European feel with great showers.=Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortab..."
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Comfortable and luxurious beds.,"The rooms are beautiful, spacious, and quite.=Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.=The beds are amazingly comfortable, with top end linens.=These quiet and spacious and luxurious rooms had wonderful European feel with great showers.=Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortab..."
...,...,...,...,...,...,...
193,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Memorable going away party.,"The room was fairly basic post-remodeling, but it was roomy and had plenty of modern charm, with incredibly comfy beds and a personal balcony to lounge on.=Amenities include an in-room safe, a small refrigerator, and a fireplace.=Privacy might be a concern in the bathrooms that have sliding glass doors, if one isn't travelling alone.=The rooms were remodeled with a modern, basic charm.=However, they have an inviting view of the beach and a balcony with chairs.=There was little privacy in the..."
194,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Perfect evening and morning.,"The room was fairly basic post-remodeling, but it was roomy and had plenty of modern charm, with incredibly comfy beds and a personal balcony to lounge on.=Amenities include an in-room safe, a small refrigerator, and a fireplace.=Privacy might be a concern in the bathrooms that have sliding glass doors, if one isn't

In [20]:
cands, refs= preprocess_text(softp_data, metrics = "softPrecision")

#BART score cannot be processed for different numbers of reference sentences, so
#check for the maximum number of reference sentences and match the size.
#We fill in the None for the missing sentences because we only pick one of the 
#maximum values and not the average, so there is no impact on performance

refs = balance_ref_num(refs)        

# generation scores from the first list of texts to the second list of texts.
P = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4) # agg means aggregation, can be mean or max

#mapping the score to (0,1]
P_average = math.tanh(math.exp((mean(P))/2+1.3))
#P_average = math.tanh(mean(P)) + 1
#P_average = math.exp(mean(P))

In [20]:
P_average

0.6905645553726367

In [23]:
P_average

0.7443144278583115

### SoftRecall

In [23]:
softr_data = merged_df.groupby(['category', 'entity_id', 'topic', 'stance', 'key_point_given'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point'].tolist())])).reset_index(name='multi_cands')
softr_data

,category,entity_id,topic,stance,key_point_given,multi_cands
0,Hotel,156564,Wedgewood Hotel & Spa,positive,All the staff were extremely helpful and polite.,A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
1,Hotel,156564,Wedgewood Hotel & Spa,positive,"All the staff were extremely helpful, polite, and always had a smile.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
2,Hotel,156564,Wedgewood Hotel & Spa,positive,"Amenities included flat screen television sets, DVD players.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
3,Hotel,156564,Wedgewood Hotel & Spa,positive,"Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortable stay.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Another nice touch was the chocolates and cookies with turndown.,A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
...,...,...,...,...,...,...
350,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,There was little privacy in the bathroom.,A home away from home.=California is a must-do.=Clean and comfortable rooms.=Close proximity to the beach.=Comfortable and luxurious towels.=Comfortable beds and pillows.=Convenient parking options available.=Delicious breakfast and great cocktails.=Delicious food at Hash restaurant.=Enjoyable and pleasant stay.=Eugene provided exceptional service.=Excellent location with great service.=Friendly and helpful staff.=Friendly bartender with great recommendations.=Funky and modern decor.=Good va...
351,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,"There was oatmeal, great wheat toast, freshberries and a tasty corned beef hash and various other items",A hom

In [24]:
cands, refs= preprocess_text(softr_data, metrics = "softRecall")

refs = balance_ref_num(refs)

R = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4)

#mapping the score to (0,1]
R_average = math.tanh(math.exp((mean(R)/2)+1.3)) 
#R_average = (0.25 * mean(R)) + 1
#R_average = math.exp(mean(R))

In [15]:
R_average

0.7077926347982703

In [27]:
R_average

0.6626422886324295

### F1

In [26]:
result = softF1(P_average, R_average)

In [17]:
result

0.6979538059491077

In [29]:
result

0.7011078736925943

In [17]:
softp_data

,ID,key_point,key_point_given,topic,stance,multi_cands
0,0,should not be mandatory because there are many side effects. Routine child vaccinations should not be mandatory because there are many side effects,"Routine child vaccinations, or their side effects, are dangerous",Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
1,1,should not be mandatory,Mandatory vaccination contradicts basic rights,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
2,2,is a violation of human rights,The parents and not the state should decide,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
3,3,vaccination is not urgent because children don't suffer from symptoms,Routine child vaccinations are not necessary to keep children healthy,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
4,4,is one of the most effective public health interventions for saving lives and promoting good health,Routine child vaccinations are effective,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
5,5,saves lives,Child vaccination saves lives,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
6,6,vaccinations are necessary to ensure that the population are protected from communicable diseases,Routine child vaccinations are necessary to protect others,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
7,7,should be mandatory to avoid spreading the virus. Routine child vaccinations should be mandatory to avoid spreading the virus,Routine child vaccinations should be mandatory to prevent virus/disease spreading,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
8,8,prevents diseases in children. Vaccines prevent diseases from childhood,Children should not suffer from preventable diseases,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer fro

## BLEURTScore

In [25]:
softp_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
1,Beautiful and spacious bathroom.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere."
2,Beautiful and spacious bathroom.,Every room was tastefully clean and spotless.
3,Beautiful and spacious bathroom.,The rooms (beds and showers included) were very comfortable and filled with first class amenities.
4,Beautiful and spacious bathroom.,"The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food."
5,Beautiful and spacious bathroom.,"The hotel is close to a movie theater, shops, and restaurants on Robson St."
...,...,...
3398,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort."
3399,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash."
3400,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty."
3401,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast."


In [26]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [28]:
import tensorflow as tf

In [29]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [31]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9709736529553824590
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1620072431990812415
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [32]:
with tf.device('/gpu:0'):
  result = calculatingScore(references, candidates)
  df_compare_precision["BLEURT Score"] = result

  #After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
  df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
  #take average of all best scores as the soft precision score.
  P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

### SoftPrecision

In [21]:
softp_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
1,Beautiful and spacious bathroom.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere."
2,Beautiful and spacious bathroom.,Every room was tastefully clean and spotless.
3,Beautiful and spacious bathroom.,The rooms (beds and showers included) were very comfortable and filled with first class amenities.
4,Beautiful and spacious bathroom.,"The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food."
5,Beautiful and spacious bathroom.,"The hotel is close to a movie theater, shops, and restaurants on Robson St."
...,...,...
3398,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort."
3399,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash."
3400,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty."
3401,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast."


In [22]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [23]:
import tensorflow as tf

In [24]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-02-03 18:13:14.783059: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:14.805999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:14.806189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18435889184579564257
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22356492288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5549650304347576173
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-02-03 18:13:16.587623: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:16.587776: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:16.587907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [26]:
result = calculatingScore(references, candidates)
df_compare_precision["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
#take average of all best scores as the soft precision score.
P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2024-02-03 18:13:24.502151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:24.502354: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:24.502494: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [27]:
P_average

0.6124507463445851

### SoftRecall

In [33]:
softr_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point_given'])
df_compare_recall = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point_given': 'candidate', 'key_point': 'reference'})
df_compare_recall

,reference,candidate
36,Beautiful and spacious bathroom.,All the staff were extremely helpful and polite.
95,Beautiful and spacious suites.,All the staff were extremely helpful and polite.
154,Comfortable and luxurious beds.,All the staff were extremely helpful and polite.
213,Delicious cookies provided during turndown service.,All the staff were extremely helpful and polite.
272,Delicious meals and impressive breakfast.,All the staff were extremely helpful and polite.
...,...,...
3133,Excellent location with great service.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3193,Friendly and helpful staff.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3253,Highly recommended hotel with great experience.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3313,Stunning ocean views.,Very close to Venice Beach and a long-ish walk to Santa Monica.


In [34]:
candidates = df_compare_recall['candidate']
references = df_compare_recall['reference']

In [35]:
import tensorflow as tf

In [36]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [37]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3616641842038152970
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22356492288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2208089500298092512
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-02-03 18:21:53.444547: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:21:53.444746: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:21:53.444884: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [38]:
result = calculatingScore(references, candidates)
df_compare_recall["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair. 
df_bestkp_pair_recall = df_compare_recall.loc[df_compare_recall.groupby(["candidate"])["BLEURT Score"].idxmax()]

#take average of all best scores as the soft precision score.
R_average = df_bestkp_pair_recall["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint BLEURT-20.


INFO:tensorflow:Reading checkpoint BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [39]:
R_average

0.5088073847170212

### F1

In [40]:
result = softF1(P_average, R_average)

In [41]:
result

0.555838934644844

## ROUGE

In [51]:
gt_gold_kp = merged_df

In [52]:
gt_gold_kp

,category,entity_id,topic,stance,key_point,key_point_given,P,R
1,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The rooms are beautiful, spacious, and quite.",0.475561,0.475597
2,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.",0.367027,0.367069
3,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The beds are amazingly comfortable, with top end linens.",0.305698,0.305746
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,These quiet and spacious and luxurious rooms had wonderful European feel with great showers.,0.523816,0.523848
5,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortable stay.",0.342050,0.342096
...,...,...,...,...,...,...,...,...
2331,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort.",0.447668,0.447706
2332,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash.",0.227394,0.227448
2333,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty.",0.342214,0.342261
2334,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast.",0.398586,0.398628


In [53]:
predictions, references = [], []
for topic in sorted(gt_gold_kp['topic'].unique()):
    for stance in ['negative', 'positive']:
        kps = gt_gold_kp.loc[(gt_gold_kp['topic']==topic) & (gt_gold_kp['stance']==stance), 'key_point'].unique().tolist()
        gold_kps = gt_gold_kp.loc[(gt_gold_kp['topic']==topic) & (gt_gold_kp['stance']==stance), 'key_point_given'].unique().tolist()
        if len(kps) > 0 and len(gold_kps) > 0:
            predictions.append(kps)
            references.append(gold_kps)

In [40]:
compute_rouge_max(predictions, references)

Rouge 1: 0.648
Rouge 2: 0.364
Rouge L: 0.51
